# Portafolio de Crypto

In [3]:
import pandas as pd
import yfinance as yf

from datetime import datetime

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

(CVXPY) Mar 14 09:57:32 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 14 09:57:32 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


Parece que Yahoo Finance selecciona criptomonedas basándose en **capitalización de mercado, liquidez y demanda de los usuarios**. Estas 28 criptomonedas incluyen:  

1. **Las más grandes por capitalización de mercado** (BTC, ETH, USDT, BNB, USDC, XRP, ADA, SOL, DOGE).  
2. **Stablecoins relevantes** (USDT, USDC, DAI, USDS).  
3. **Tokens de plataformas blockchain** (TRX, TON, MATIC, DOT, AVAX, STETH, WSTETH, SUI).  
4. **Activos envueltos (wrapped assets)** para interoperabilidad (WBTC, WTRX, WSTETH).  
5. **Criptos con adopción en pagos o DeFi** (LTC, LINK, XLM, HBAR, BCH).  
6. **Memecoins populares** (DOGE, SHIB).  
7. **Tokens de exchanges** (LEO).  

Yahoo Finance probablemente no lista **todas las criptomonedas** porque:  
- **Filtra por popularidad y uso** (no incluye proyectos pequeños).  
- **Depende de proveedores de datos** que ofrecen precios y volumen.  
- **Evita listar tokens de baja liquidez** para reducir manipulación.  

Si buscas criptos fuera de esta lista, necesitarás APIs más completas como CoinGecko o CoinMarketCap.

In [4]:
cryptos = [
    "BTC-USD",    # Bitcoin
    "ETH-USD",    # Ethereum
    "USDT-USD",   # Tether
    "BNB-USD",    # Binance Coin
    "USDC-USD",   # USD Coin
    "XRP-USD",    # XRP
    "ADA-USD",    # Cardano
    "SOL-USD",    # Solana
    "DOGE-USD",   # Dogecoin
    "TRX-USD",    # Tron
    "TON-USD",    # Toncoin
    "MATIC-USD",  # Polygon
    "DOT-USD",    # Polkadot
    "LTC-USD",    # Litecoin
    "AVAX-USD",   # Avalanche
    "WBTC-USD",   # Wrapped Bitcoin
    "DAI-USD",    # DAI
    "WTRX-USD",   # Wrapped TRON
    "STETH-USD",  # Lido Staked ETH
    "LINK-USD",   # Chainlink
    "LEO-USD",    # UNUS SED LEO
    "XLM-USD",    # Stellar
    "WSTETH-USD", # Lido wstETH
    "HBAR-USD",   # Hedera
    "USDS33039-USD", # USDS
    "SHIB-USD",   # Shiba Inu
    "SUI20947-USD", # Sui
    "BCH-USD"     # Bitcoin Cash
]

In [5]:
adj_close_list = []

for crypto in cryptos:
    stock = yf.Ticker(crypto)
    hist = stock.history(period="max")
    adj_close_list.append(hist['Close'].rename(crypto))

    # Concatenate all data into a single DataFrame
    data = pd.concat(adj_close_list, axis=1)

    # Save the adjusted close prices to a CSV file
    data.to_csv("crypto_adj_close.csv", index=True)

In [4]:
data.dropna(inplace=True) # Inplace=True means that the changes are made to the original DataFrame

In [5]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(data)
s = risk_models.sample_cov(data)

In [6]:
mu

BTC-USD      0.457962
ETH-USD      0.491725
USDT-USD    -0.000249
BNB-USD      0.739113
USDC-USD    -0.000052
XRP-USD      0.470542
ADA-USD      0.520508
SOL-USD      1.014671
DOGE-USD     1.264650
TRX-USD      0.492731
TON-USD     -0.101069
MATIC-USD    0.737916
DOT-USD      0.126758
LTC-USD      0.169909
AVAX-USD     0.439322
WBTC-USD     0.457218
DAI-USD     -0.001905
dtype: float64

In [7]:
ef = EfficientFrontier(mu, s)

In [8]:
weights = ef.max_sharpe()

In [9]:
weights

OrderedDict([('BTC-USD', 0.0),
             ('ETH-USD', 0.0),
             ('USDT-USD', 0.0),
             ('BNB-USD', 0.4329051861376632),
             ('USDC-USD', 0.0),
             ('XRP-USD', 0.0),
             ('ADA-USD', 0.0),
             ('SOL-USD', 0.3380396719923829),
             ('DOGE-USD', 0.1370702116581472),
             ('TRX-USD', 0.0292929889857963),
             ('TON-USD', 0.0),
             ('MATIC-USD', 0.0),
             ('DOT-USD', 0.0),
             ('LTC-USD', 0.0),
             ('AVAX-USD', 0.0),
             ('WBTC-USD', 0.0626919412260105),
             ('DAI-USD', 0.0)])

In [10]:
# get how many money we will invest
total_investment = 1000

# get the amount of money we will invest in each crypto
amounts = ef.clean_weights()
amounts

OrderedDict([('BTC-USD', 0.0),
             ('ETH-USD', 0.0),
             ('USDT-USD', 0.0),
             ('BNB-USD', 0.43291),
             ('USDC-USD', 0.0),
             ('XRP-USD', 0.0),
             ('ADA-USD', 0.0),
             ('SOL-USD', 0.33804),
             ('DOGE-USD', 0.13707),
             ('TRX-USD', 0.02929),
             ('TON-USD', 0.0),
             ('MATIC-USD', 0.0),
             ('DOT-USD', 0.0),
             ('LTC-USD', 0.0),
             ('AVAX-USD', 0.0),
             ('WBTC-USD', 0.06269),
             ('DAI-USD', 0.0)])

In [11]:
for crypto, weight in weights.items():
    amounts[crypto] = weight * total_investment

amounts

OrderedDict([('BTC-USD', 0.0),
             ('ETH-USD', 0.0),
             ('USDT-USD', 0.0),
             ('BNB-USD', 432.90518613766324),
             ('USDC-USD', 0.0),
             ('XRP-USD', 0.0),
             ('ADA-USD', 0.0),
             ('SOL-USD', 338.03967199238286),
             ('DOGE-USD', 137.0702116581472),
             ('TRX-USD', 29.292988985796303),
             ('TON-USD', 0.0),
             ('MATIC-USD', 0.0),
             ('DOT-USD', 0.0),
             ('LTC-USD', 0.0),
             ('AVAX-USD', 0.0),
             ('WBTC-USD', 62.691941226010506),
             ('DAI-USD', 0.0)])

In [12]:
print("Amounts to invest in each crypto:")
for crypto, amount in amounts.items():
    if amount > 0:
        print(f"{crypto}: ${amount:.2f}")

Amounts to invest in each crypto:
BNB-USD: $432.91
SOL-USD: $338.04
DOGE-USD: $137.07
TRX-USD: $29.29
WBTC-USD: $62.69


In [13]:
ef.portfolio_performance(verbose=True)

Expected annual return: 87.9%
Annual volatility: 72.6%
Sharpe Ratio: 1.18


(0.8794080456936608, 0.7256767594288964, 1.1842849237310702)